# Download All PDFs
Nicolas Chan, 11/2/2017

In [ ]:
# Configuration
all_files_list = '/global/scratch/groups/dh/aanderson/all_files.txt'
destination = '/global/scratch/groups/dh/aanderson/all_pdfs/'
start_index, end_index = 0, 100

In [ ]:
# bDrive Authorization
import os
import sys
import codecs
import httplib2
from googleapiclient.http import MediaIoBaseDownload
from googleapiclient.http import MediaFileUpload
from apiclient import discovery, errors
from oauth2client import client
from oauth2client import tools
from oauth2client.file import Storage

SCOPES = 'https://www.googleapis.com/auth/drive'
CLIENT_SECRET_FILE = 'client_secret.json'
APPLICATION_NAME = 'gDriveConnect'

import argparse
parser = argparse.ArgumentParser(parents=[tools.argparser])
parser.add_argument('-f', help=argparse.SUPPRESS)

flags = parser.parse_known_args()[0]
flags.noauth_local_webserver = True

def get_credentials():
    
    home_dir = os.path.expanduser('~')
    credential_dir = os.path.join(home_dir, '.credentials')
    if not os.path.exists(credential_dir):
        os.makedirs(credential_dir)
    credential_path = os.path.join(credential_dir, 'gDriveConnect.json')
    
    store = Storage(credential_path)    
    credentials = store.get()
    
    if not credentials or credentials.invalid:
        flow = client.flow_from_clientsecrets(CLIENT_SECRET_FILE, SCOPES)
        flow.user_agent = APPLICATION_NAME
        if flags:
            credentials = tools.run_flow(flow, store, flags)
        else: # Needed only for compatibility with Python 2.6
            credentials = tools.run(flow, store)
        print('Storing credentials to ' + credential_path)
        
    return credentials

credentials = get_credentials()
print('credential_path:', credentials)
http = credentials.authorize(httplib2.Http())
service = discovery.build('drive', 'v3', http=http)

In [ ]:
import io
def download_file(google_id, destination):
    """Downloads a file from Google Drive"""
    
    request = service.files().get_media(fileId=google_id)
    fh = io.BytesIO()
    downloader = MediaIoBaseDownload(fh, request)
    
    done = False
    while done is False:
        try:
            status, done = downloader.next_chunk()
            sys.stdout.write('.')
        except errors.HttpError as error :
            print('Error file:', value, '   id:', key)
            print('An error occurred pulling the next chunk:', error)
            break

    fh.seek(0)

    print('\nwriting:', destination)
    with open( destination, 'wb',) as f2:
        f2.write(fh.getvalue())
        f2.close()

    fh.close()

In [ ]:
with open(all_files_list, 'rt', encoding='utf-8') as f:
    line_number = 0
    for line in f:
        if line_number > end_index:
            break
        if line_number >= start_index:
            file = line.split(';')[-1].strip('\n')
            file_path = destination + file + '.pdf'
            if os.path.isfile(file_path):
                print(file, 'already exists, skipping')
            else:
                try:
                    print('Downloading', file)
                    download_file(file, file_path)
                except:
                    print('Failed to download', file)
                    !echo $file >> /global/scratch/groups/dh/aanderson/failed_files_download_all.txt
        line_number += 1